In [ ]:
from __future__ import division
import re
import subprocess as sp
# import gurobipy as gp
import sage.all as sg
from sage.all import libgap as lg
import time
import os
import copy
import cplex as cp
import math
import scipy
import fractions
import random
from inner_build_lp_loop import lift_cut_cy_average, aggregate_A_mat_cy, lift_cut_cy_single, aggregate_A_mat_average_cy

In [ ]:
class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None
        self._obj_change_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")
        if self._obj_change_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()
        self._obj_change_time = time.perf_counter()
        
    def get_run_time(self):
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
        return time.perf_counter() - self._start_time
    
    def get_obj_time(self):
        if self._obj_change_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
        return time.perf_counter() - self._obj_change_time
    
    def reset_obj_time(self):
        if self._obj_change_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
        self._obj_change_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        self._obj_change_time = None

class orbital_partition:
    def __init__(self, mdl, orbits):
        t_0 = time.perf_counter()
        num_col_orbits = 0
        num_row_orbits = 0
        self.num_col_orbits = 0
        self.num_row_orbits = 0
        self.orbit_col = []
        self.orbit_col_start = [0]
        self.orbit_row = []
        self.orbit_row_start = [0]
        self.col_orbit = {}
        self.row_orbit = {}
        self.col_orbit_rep = {}
        self.row_orbit_rep = {}
        self.col_orbit_size = {}
        self.row_orbit_size = {}
#         print(orbits)
#         input()
        for orbit in orbits:
            if orbit[0] < mdl.variables.get_num():
                for el in orbit:
                    self.orbit_col.append(el)
                    self.col_orbit[int(el)] = num_col_orbits
                    if self.col_orbit_size.get(num_col_orbits) == None:
                        self.col_orbit_size[num_col_orbits] = 0
                    self.col_orbit_size[num_col_orbits] += 1
                self.orbit_col_start.append(len(self.orbit_col))
                self.col_orbit_rep[num_col_orbits] = int(orbit[0])
                num_col_orbits += 1
                self.num_col_orbits += 1
            else:
                for el in orbit:
                    self.orbit_row.append(el - mdl.variables.get_num())
                    self.row_orbit[int(el - mdl.variables.get_num())] = num_row_orbits
                    if self.row_orbit_size.get(num_row_orbits) == None:
                        self.row_orbit_size[num_row_orbits] = 0
                    self.row_orbit_size[num_row_orbits] += 1
                self.orbit_row_start.append(len(self.orbit_row))
                self.row_orbit_rep[num_row_orbits] = int(orbit[0] - mdl.variables.get_num())
                num_row_orbits += 1
                self.num_row_orbits += 1
        t_n = time.perf_counter() - t_0
        print("Time to create orbit part: %.2f" % t_n)
    def get_num_orbits(self):
        return self.num_col_orbits, self.num_row_orbits
    def get_orbits(self):
        return self.col_orbit, self.row_orbit
    def get_orbit_reps(self):
        return self.col_orbit_rep, self.row_orbit_rep
    def get_orbit_sizes(self):
        return self.col_orbit_size, self.row_orbit_size

class orbital_cut_generation_stats:
    def __init__(self):
        self.lift_iter = 0
        self.lift_cuts = 0

In [13]:
def cplex_read_model(model_file):
    return cp.Cplex(model_file)

def aggregate_A_mat(mdl, orbits):
    agg_mdl = aggregate_A_mat_average_cy(mdl, orbits)
    return agg_mdl

def is_zero_cut(cut):
    for val in cut[:-1]:
        if abs(val) > 1e-6:
            return False
    return True

def is_integral_gomory(cut):
    for val in cut[:-1]:
        if abs(val - round(val)) > 1e-6:
            return False
    return True

def is_almost_integral(mdl):
    for sol_val in mdl.solution.get_values():
        if (abs(round(sol_val) - sol_val) > .01):
            return False
    return True

def most_fractional(base_idx, base_val, fract_rows, skip):
    min_f = 1
    min_row = -1
    for i_row in fract_rows:
        if i_row in skip:
            continue
        b_val = base_val[i_row]
        f = b_val - math.floor(b_val)
        delta = abs(.5 - f)
        if delta < min_f:
            min_f = delta
            min_row = i_row
    return min_row, min_f
            
def solve_cplex(mdl):
#     agg_mdl.parameters.lpmethod.set(agg_mdl.parameters.lpmethod.values.primal)
    mdl.set_problem_type(mdl.problem_type.LP)
    mdl.solve()
    
def get_fractional_rows(base_idx, base_val):
    fractional_rows = []
    for i_row in range(len(base_idx)):
        i_col = base_idx[i_row]
        if i_col < 0:
            continue
        if abs(round(base_val[i_row]) - base_val[i_row]) > 1e-6:
            fractional_rows.append(i_row)
    return fractional_rows
    
def generate_cuts(full_mdl, mdl, group, first_group, orbits, hash_table, node_table, p_obj, iterate,
                  total_full_cut, total_agg_cut, cut_reps, cut_rhs, col_orbit_rep_by_level, col_orbit_by_level, 
                  solution_by_level, timer, bb, check_sol):
    it = 0
    level = None
    feasible = True
    time_out = 0
    optimal = 0
    return_orbits = []
    cut_test = {}
    row_cut = {}
    return_cut_orbit_sizes = {}
    num_full_cut = 0
    num_agg_cut = 0
    this_round_num_col = mdl.variables.get_num()
    this_round_num_row = mdl.linear_constraints.get_num()
    stop_row = this_round_num_row - total_agg_cut
    next_round = 0
    restart_opt_cut = 0
    skip_row = []
#     mdl.solution.write("cplex_agg_solution_before_cuts.txt")
#     mdl.write("cut_added_agg_model_cplex_before.lp")
#     print("New solution before cut generation")
    agg_cut_node = this_round_num_col + this_round_num_row
#     while it < 10:
    while True:
        tot_agg_cut = []
        tot_agg_cut_sense = []
        tot_agg_cut_rhs = []
        tot_agg_cut_name = []
        if time_out:
            print("\nTime limit reached at level %d.\n" % iterate)
#             input()
            break
        if optimal:
            print("\nBest known bound achieved at level %d.\n" % iterate)
#             input()
            break
        if not feasible:
            print("\nSolution is not previously almost integral at level %d.\n" % level)
#             input()
            break
        if is_solution_integral(mdl):
            print("\nSolution is integral at level %d.\n" % iterate)
#             input()
            break
        if is_almost_integral(mdl):
            print("\nSolution is almost integral at level %d\n" % iterate)
#             input()
            break
        p_obj = mdl.solution.get_objective_value()
        num_col = mdl.variables.get_num()
        num_row = mdl.linear_constraints.get_num()
        num_tot = num_col + num_row
#         full_num_col = full_mdl.variables.get_num()
#         full_num_row = full_mdl.linear_constraints.get_num()
#         full_num_tot = full_num_col + full_num_row
        row_base_idx, row_base_val = mdl.solution.basis.get_header()
        gomory_rows = get_fractional_rows(row_base_idx, row_base_val)
#         most_fract_row, fract_delta = most_fractional(row_base_idx, row_base_val, gomory_rows, skip_row)
#         print(most_fract_row)
#         input()
#         if (most_fract_row < 0):
#             break
        for fract_row in gomory_rows:
            full_num_col = full_mdl.variables.get_num()
            full_num_row = full_mdl.linear_constraints.get_num()
            full_num_tot = full_num_col + full_num_row
            reduced_row = (mdl.solution.advanced.binvarow(fract_row) + 
                           mdl.solution.advanced.binvrow(fract_row))
    #         print(reduced_row, "=", row_base_val[fract_row])
    #         input()
            for i_slack in range(num_col, num_tot):
                sense = mdl.linear_constraints.get_senses(i_slack - num_col)
                if sense == "G":
                    reduced_row[i_slack] *= -1
            for i_slack in range(num_col, num_tot):
                if orbits.row_orbit_size.get(i_slack - num_col) is None:
                    row_slack_scale = return_cut_orbit_sizes.get(i_slack)
                else:
                    row_slack_scale = orbits.row_orbit_size.get(i_slack - num_col)
                reduced_row[i_slack] /= row_slack_scale
    #         print(reduced_row, "=", row_base_val[fract_row])
    #         input()
            for i_col in range(num_tot):
                reduced_row[i_col] -= math.floor(reduced_row[i_col])
            row_base_val[fract_row] -= math.floor(row_base_val[fract_row])
    #         print(reduced_row, "=", row_base_val[fract_row])
    #         input()
            for i_slack in range(num_col, num_tot):
                value = {}
                if (abs(reduced_row[i_slack] - 0)) < 1e-6:
                    continue
                sense = mdl.linear_constraints.get_senses(i_slack - num_col)
                if orbits.row_orbit_size.get(i_slack - num_col) is None:
                    row_slack_scale = return_cut_orbit_sizes.get(i_slack)
                else:
                    row_slack_scale =  orbits.row_orbit_size.get(i_slack - num_col)
                if sense == "G":
                    slack_coeff = -1 * row_slack_scale
                else:
                    slack_coeff = 1 * row_slack_scale
                a_idx, a_val = mdl.linear_constraints.get_rows(i_slack - num_col).unpack()
                a_rhs = mdl.linear_constraints.get_rhs(i_slack - num_col)
                for idx, val in zip(a_idx, a_val):
                    reduced_row[idx] += reduced_row[i_slack] * slack_coeff * -val
                    if abs(round(reduced_row[idx]) - reduced_row[idx]) < 1e-6:
                        reduced_row[idx] = round(reduced_row[idx])
    #                 print(reduced_row, "=", row_base_val[fract_row])
    #                 input()
                row_base_val[fract_row] += -1 * reduced_row[i_slack] * slack_coeff * a_rhs
    #             print(reduced_row, "=", row_base_val[fract_row])
    #             input()
                if abs(round(row_base_val[fract_row]) - row_base_val[fract_row]) < 1e-6:
                    row_base_val[fract_row] = float(round(row_base_val[fract_row]))
                reduced_row[i_slack] = 0
            del reduced_row[num_col:len(reduced_row)]
            reduced_row.append(row_base_val[fract_row])
            numerically_stable(reduced_row)
            reduced_row = [sg.Rational(val) for val in reduced_row]
    #         print(reduced_row)
    #         input()
            if (is_zero_cut(reduced_row)):
                skip_row.append(fract_row)
#                 print("zero")
#                 input()
                continue
            cut_check = ",".join(str(el) for el in reduced_row)
            if cut_test.get(cut_check) is not None:
                skip_row.append(fract_row)
#                 print(cut_test)
#                 print(cut_check)
#                 print("cut exists already")
#                 input()
                continue
            cut_test[cut_check] = 1
            if (is_integral_gomory(reduced_row)):
                reduced_row[-1] = sg.Rational(round(reduced_row[-1]))
    #         print(reduced_row)
    #         input()
            # GENERATE GOMORY CUT ORBITS AT AGGREGATE AND EXTENDED LEVELS
            (new_orbits, new_orbit_sizes, agg_cuts, agg_cuts_sense, agg_cuts_rhs, agg_cuts_name) = lift_cut_cy_average(
                reduced_row, group, first_group,orbits, full_num_tot, hash_table, node_table, full_mdl, mdl, cut_reps, 
                cut_rhs, agg_cut_node)
            tot_agg_cut += agg_cuts
            tot_agg_cut_sense += agg_cuts_sense
            tot_agg_cut_rhs += agg_cuts_rhs
            tot_agg_cut_name += agg_cuts_name
            num_agg_cut += len(new_orbits)
            agg_cut_node += len(new_orbits)
            num_full_cut += (full_mdl.linear_constraints.get_num() - full_num_row)
            return_orbits += new_orbits
            return_cut_orbit_sizes.update(new_orbit_sizes)
            if (num_full_cut + total_full_cut > 400000):
                break
            if timer[0].get_run_time() > 10800:
                time_out = 1
                break
#             print(return_orbits)
#             print(return_cut_orbit_sizes)
#             input()
#         print(tot_agg_cut)
#         input()
        if len(tot_agg_cut) == 0:
            print("No new cuts could be generated because they exist already")
            if check_sol:
                record_previous_solution(mdl, solution_by_level, iterate)
                feasible, level = is_solution_level_feasible(col_orbit_rep_by_level, col_orbit_by_level, 
                                                         solution_by_level, iterate)
            break
        mdl.linear_constraints.add(lin_expr = tot_agg_cut, senses = tot_agg_cut_sense,
                                   rhs = tot_agg_cut_rhs, names = tot_agg_cut_name)
#         cut_add_mps = "./cut_added_model_cplex.mps"
#         cut_add_lp = "./cut_added_model_cplex.lp"
        cut_add_agg_lp = "./cut_added_agg_model_cplex_after.lp"
#         full_mdl.write(cut_add_lp)
#         full_mdl.write(cut_add_mps)
        mdl.write(cut_add_agg_lp)
#         input()
        solve_cplex(mdl)
        c_obj = copy.deepcopy(mdl.solution.get_objective_value())
        mdl.solution.write("cplex_agg_solution_after_cuts.txt")
        print("New_solution")
        if abs(c_obj - p_obj > 1e-6):
#             print(timer[0].get_obj_time())
            timer[0].reset_obj_time()
#             print(timer[0].get_obj_time())
#             input()
            opt_cut_orb, n_opt_full, n_opt_agg = add_opt_cut(full_mdl, c_obj, mdl, cut_test, 
                                                             cut_reps, cut_rhs, hash_table, node_table)
            if not n_opt_full:
                continue
            num_agg_cut += n_opt_agg
            agg_cut_node += n_opt_agg
            num_full_cut += n_opt_full
            return_orbits += opt_cut_orb
            orb_node = (mdl.variables.get_num() + 
                        mdl.linear_constraints.get_num() - 1)
            return_cut_orbit_sizes.update({orb_node: 1})
            mdl.write(cut_add_agg_lp)
            solve_cplex(mdl)
            mdl.solution.write("cplex_agg_solution_after_cuts.txt")
            print("New Solution")
#             input()
            c_obj = copy.deepcopy(mdl.solution.get_objective_value())
            if check_sol:
                record_previous_solution(mdl, solution_by_level, iterate)
                feasible, level = is_solution_level_feasible(col_orbit_rep_by_level, col_orbit_by_level, 
                                                         solution_by_level, iterate)
            if (num_full_cut + total_full_cut > 400000):
                break
#             continue
#         p_obj = copy.deepcopy(c_obj)
#         skip_row.append(fract_row)
        if check_sol:
                record_previous_solution(mdl, solution_by_level, iterate)
                feasible, level = is_solution_level_feasible(col_orbit_rep_by_level, col_orbit_by_level, 
                                                         solution_by_level, iterate)
        if (num_full_cut + total_full_cut > 400000):
                break
        if abs(bb - c_obj) < 1e-6:
            optimal = 1
        it += 1
        if timer[0].get_run_time() > 14400:
            time_out = 1
        if timer[0].get_obj_time() > 3600:
            time_out = 1
    input()
    return return_orbits, num_full_cut, num_agg_cut, feasible, level, time_out, optimal

def generate_cuts_gmi(full_mdl, mdl, group, first_group, orbits, hash_table, node_table, p_obj, iterate,
                  total_full_cut, total_agg_cut, cut_reps, cut_rhs, col_orbit_rep_by_level, col_orbit_by_level, 
                  solution_by_level, timer, bb, check_sol):
    it = 0
    level = None
    feasible = True
    time_out = 0
    optimal = 0
    integral = 0
    almost_integral = 0
    cut_limit = 0
    no_new_cuts = 0
    return_orbits = []
    cut_test = {}
    row_cut = {}
    return_cut_orbit_sizes = {}
    num_full_cut = 0
    num_agg_cut = 0
    this_round_num_col = mdl.variables.get_num()
    this_round_num_row = mdl.linear_constraints.get_num()
    stop_row = this_round_num_row - total_agg_cut
    next_round = 0
    restart_opt_cut = 0
    skip_row = []
    mdl.solution.write("cplex_agg_solution_before_cuts.txt")
    mdl.write("cut_added_agg_model_cplex_before.lp")
    print("New solution before cut generation")
    input()
    agg_cut_node = this_round_num_col + this_round_num_row
#     while it < 10:
    while True:
        tot_agg_cut = []
        tot_agg_cut_sense = []
        tot_agg_cut_rhs = []
        tot_agg_cut_name = []
        if time_out:
            print("\nTime limit reached at level %d.\n" % iterate)
#             input()
            break
        if optimal:
            print("\nBest known bound achieved at level %d.\n" % iterate)
#             input()
            break
        if no_new_cuts:
            print("\nNo new cuts can be generated right now at level %d" % iterate)
            feasible = 1
            break
        if not feasible:
            print("\nSolution is not previously almost integral at level %d.\n" % level)
#             input()
            break
        if integral:
            print("\nSolution is integral at level %d.\n" % iterate)
#             input()
            break
        if almost_integral:
            print("\nSolution is almost integral at level %d\n" % iterate)
#             input()
            break
        if cut_limit:
            print("\nCut limit reached at level %d\n" % iterate)
#             input()
            break
        p_obj = mdl.solution.get_objective_value()
        num_col = mdl.variables.get_num()
        num_row = mdl.linear_constraints.get_num()
        num_tot = num_col + num_row
#         full_num_col = full_mdl.variables.get_num()
#         full_num_row = full_mdl.linear_constraints.get_num()
#         full_num_tot = full_num_col + full_num_row
        row_base_idx, row_base_val = mdl.solution.basis.get_header()
        gomory_rows = get_fractional_rows(row_base_idx, row_base_val)
#         most_fract_row, fract_delta = most_fractional(row_base_idx, row_base_val, gomory_rows, skip_row)
#         print(most_fract_row)
#         input()
#         if (most_fract_row < 0):
#             break
#         print("Total cuts so far:", total_full_cut)
        for fract_row in gomory_rows:
            t_0 = time.perf_counter()
            full_num_col = full_mdl.variables.get_num()
            full_num_row = full_mdl.linear_constraints.get_num()
            full_num_tot = full_num_col + full_num_row
            reduced_row = (mdl.solution.advanced.binvarow(fract_row) + 
                           mdl.solution.advanced.binvrow(fract_row))
    #         print(reduced_row, "=", row_base_val[fract_row])
    #         input()
            for i_slack in range(num_col, num_tot):
                sense = mdl.linear_constraints.get_senses(i_slack - num_col)
                if sense == "G":
                    reduced_row[i_slack] *= -1
            for i_slack in range(num_col, num_tot):
                if orbits.row_orbit_size.get(i_slack - num_col) is None:
                    row_slack_scale = return_cut_orbit_sizes.get(i_slack)
                else:
                    row_slack_scale = orbits.row_orbit_size.get(i_slack - num_col)
                reduced_row[i_slack] /= row_slack_scale
#             print(reduced_row, "=", row_base_val[fract_row])
#             input()
            row_base_val[fract_row] -= math.floor(row_base_val[fract_row])
            for i_col in range(num_tot):
#                 if i_col == row_base_idx[fract_row]:
#                     reduced_row[i_col] = 0
                if i_col >= num_col and reduced_row[i_col] < 0:
                    reduced_row[i_col] /= -1*(1 - row_base_val[fract_row])
                elif i_col >= num_col:
                    reduced_row[i_col] /= (row_base_val[fract_row])
                elif (reduced_row[i_col] - math.floor(reduced_row[i_col])) > row_base_val[fract_row]:
                    reduced_row[i_col] = ((1 - (reduced_row[i_col] - math.floor(reduced_row[i_col])))/
                                           (1 - row_base_val[fract_row]))
                else:
                    reduced_row[i_col] = ((reduced_row[i_col] - math.floor(reduced_row[i_col]))/
                                          row_base_val[fract_row])
#                 reduced_row[i_col] -= math.floor(reduced_row[i_col])
#             print(reduced_row, "=", row_base_val[fract_row])
#             input()
            gmi_rhs = 1
            for i_slack in range(num_col, num_tot):
                value = {}
                if (abs(reduced_row[i_slack] - 0)) < 1e-6:
                    continue
                sense = mdl.linear_constraints.get_senses(i_slack - num_col)
                if orbits.row_orbit_size.get(i_slack - num_col) is None:
                    row_slack_scale = return_cut_orbit_sizes.get(i_slack)
                else:
                    row_slack_scale =  orbits.row_orbit_size.get(i_slack - num_col)
                if sense == "G":
                    slack_coeff = -1 * row_slack_scale
                else:
                    slack_coeff = 1 * row_slack_scale
                a_idx, a_val = mdl.linear_constraints.get_rows(i_slack - num_col).unpack()
                a_rhs = mdl.linear_constraints.get_rhs(i_slack - num_col)
                for idx, val in zip(a_idx, a_val):
#                     print("x_%d" % idx)
#                     print(reduced_row[i_slack])
#                     print(slack_coeff)
#                     print(-val)
#                     input()
                    reduced_row[idx] += reduced_row[i_slack] * slack_coeff * -val
                    if abs(round(reduced_row[idx]) - reduced_row[idx]) < 1e-6:
                        reduced_row[idx] = round(reduced_row[idx])
    #                 print(reduced_row, "=", row_base_val[fract_row])
    #                 input()
                row_base_val[fract_row] += -1 * reduced_row[i_slack] * slack_coeff * a_rhs
                gmi_rhs += -1 * reduced_row[i_slack] * slack_coeff * a_rhs
    #             print(reduced_row, "=", row_base_val[fract_row])
    #             input()
                if abs(round(row_base_val[fract_row]) - row_base_val[fract_row]) < 1e-6:
                    row_base_val[fract_row] = float(round(row_base_val[fract_row]))
                reduced_row[i_slack] = 0
            del reduced_row[num_col:len(reduced_row)]
            reduced_row.append(gmi_rhs)
            numerically_stable(reduced_row)
            reduced_row = [sg.Rational(val) for val in reduced_row]
#             print(reduced_row)
#             input()
            if (is_zero_cut(reduced_row)):
                skip_row.append(fract_row)
#                 print("zero")
#                 input()
                continue
            cut_check = ",".join(str(el) for el in reduced_row)
            if cut_test.get(cut_check) is not None:
                skip_row.append(fract_row)
#                 print(cut_test)
#                 print(cut_check)
#                 print("cut exists already")
#                 input()
                continue
            cut_test[cut_check] = 1
            if (is_integral_gomory(reduced_row)):
                reduced_row[-1] = sg.Rational(round(reduced_row[-1]))
    #         print(reduced_row)
    #         input()
            print("Time to generate original gomory mixed integer cut %.2f" % (time.perf_counter() - t_0))
            # GENERATE GOMORY CUT ORBITS AT AGGREGATE AND EXTENDED LEVELS
            (new_orbits, new_orbit_sizes, agg_cuts, agg_cuts_sense, agg_cuts_rhs, agg_cuts_name) = lift_cut_cy_average(
                reduced_row, group, first_group,orbits, full_num_tot, hash_table, node_table, full_mdl, mdl, cut_reps, 
                cut_rhs, agg_cut_node)
            tot_agg_cut += agg_cuts
            tot_agg_cut_sense += agg_cuts_sense
            tot_agg_cut_rhs += agg_cuts_rhs
            tot_agg_cut_name += agg_cuts_name
            num_agg_cut += len(new_orbits)
            agg_cut_node += len(new_orbits)
            num_full_cut += (full_mdl.linear_constraints.get_num() - full_num_row)
            return_orbits += new_orbits
            return_cut_orbit_sizes.update(new_orbit_sizes)
#             print("Num new full cuts:", full_mdl.linear_constraints.get_num() - full_num_row)
#             print("Combined num full cuts:", num_full_cut + total_full_cut)
#             input()
            if (num_full_cut + total_full_cut > 400000):
                break
            if timer[0].get_run_time() > 14400:
                time_out = 1
                break
#             print(return_orbits)
#             print(return_cut_orbit_sizes)
#             input()
#         print(tot_agg_cut)
#         input()
        if len(tot_agg_cut) == 0:
            no_new_cuts = 1
        mdl.linear_constraints.add(lin_expr = tot_agg_cut, senses = tot_agg_cut_sense,
                                   rhs = tot_agg_cut_rhs, names = tot_agg_cut_name)
#         cut_add_mps = "./cut_added_model_cplex.mps"
#         cut_add_lp = "./cut_added_model_cplex.lp"
        cut_add_agg_lp = "./cut_added_agg_model_cplex_after.lp"
#         full_mdl.write(cut_add_lp)
#         full_mdl.write(cut_add_mps)
#         mdl.write(cut_add_agg_lp)
#         input()
        solve_cplex(mdl)
        c_obj = copy.deepcopy(mdl.solution.get_objective_value())
#         mdl.solution.write("cplex_agg_solution_after_cuts.txt")
#         print("New_solution")
#         input()
        if abs(c_obj - p_obj > 1e-6):
#             print(timer[0].get_obj_time())
            timer[0].reset_obj_time()
#             print(timer[0].get_obj_time())
#             input()
            opt_cut_orb, n_opt_full, n_opt_agg = add_opt_cut(full_mdl, c_obj, mdl, cut_test, 
                                                             cut_reps, cut_rhs, hash_table, node_table)
            if not n_opt_full:
                continue
            num_agg_cut += n_opt_agg
            agg_cut_node += n_opt_agg
            num_full_cut += n_opt_full
            return_orbits += opt_cut_orb
            orb_node = (mdl.variables.get_num() + 
                        mdl.linear_constraints.get_num() - 1)
            return_cut_orbit_sizes.update({orb_node: 1})
#             mdl.write(cut_add_agg_lp)
            solve_cplex(mdl)
#             mdl.solution.write("cplex_agg_solution_after_cuts.txt")
#             print("New Solution")
#             input()
            c_obj = copy.deepcopy(mdl.solution.get_objective_value())
#             if check_sol:
#                 record_previous_solution(mdl, solution_by_level, iterate)
#                 feasible, level = is_solution_level_feasible(col_orbit_rep_by_level, col_orbit_by_level, 
#                                                          solution_by_level, iterate)
#             if (num_full_cut + total_full_cut > 400000):
#                 break
#             continue
#         p_obj = copy.deepcopy(c_obj)
#         skip_row.append(fract_row)
        if check_sol:
                record_previous_solution(mdl, solution_by_level, iterate)
                feasible, level = is_solution_level_feasible(col_orbit_rep_by_level, col_orbit_by_level, 
                                                         solution_by_level, iterate)
        if (num_full_cut + total_full_cut > 400000):
            cut_limit = 1
        if abs(bb - c_obj) < 1e-6:
            optimal = 1
        it += 1
        if timer[0].get_run_time() > 14400:
            time_out = 1
        if timer[0].get_obj_time() > 7200:
            time_out = 1
        if is_solution_integral(mdl):
            integral = 1
        if is_almost_integral(mdl):
            almost_integral = 1
        mdl.write(cut_add_agg_lp)
        mdl.solution.write("cplex_agg_solution_after_cuts.txt")
        print("New_solution")
        input()
#     input()
    return return_orbits, num_full_cut, num_agg_cut, feasible, level, time_out, optimal

def add_opt_cut(full_mdl, obj_val, mdl, cut_test, cut_reps, cut_rhs, hash_table, node_table):
    obj_sense = full_mdl.objective.get_sense()
    if (abs(round(obj_val) - obj_val)) < 1e-6:
        return [[]], 0, 0
    if obj_sense == 1:
        opt_cut_sense = ["G"]
        opt_cut_rhs = [math.ceil(obj_val)]
    else:
        opt_cut_sense = ["L"]
        opt_cut_rhs = [math.floor(obj_val)]
    # Aggregate model opt cut
    coeffs = mdl.objective.get_linear()
    cut_list = [el for el in coeffs]
    cut_list.append(sg.Rational(opt_cut_rhs[0]))
    numerically_stable(cut_list)
#     numerically_stable(cut_list)
    cut_list = [sg.Rational(el) for el in cut_list]
    cut_check = ",".join(str(el) for el in cut_list)
    if cut_test.get(cut_check) is not None:
        return [[]], 0, 0
    opt_cut_name = []
    opt_cut = [cp.SparsePair([i for i in range(len(coeffs))], coeffs)]
    mdl.linear_constraints.add(lin_expr = opt_cut, senses = opt_cut_sense,
                               rhs = opt_cut_rhs, names = opt_cut_name)
    # Full model opt cut
    coeffs = full_mdl.objective.get_linear()
    cut_list = [el for el in coeffs]
    cut_list.append(sg.Rational([opt_cut_rhs[0]]))
    numerically_stable(cut_list)
#     numerically_stable(cut_list)
    cut_list = [sg.Rational(el) for el in cut_list]
    cut_check = ",".join(str(el) for el in cut_list)
    opt_cut = [cp.SparsePair([i for i in range(len(coeffs))], coeffs)]
    full_mdl.linear_constraints.add(lin_expr = opt_cut, senses = opt_cut_sense,
                                    rhs = opt_cut_rhs, names = opt_cut_name)
    cut_node = full_mdl.variables.get_num() + full_mdl.linear_constraints.get_num() - 1
    cut_reps[cut_node] = [lg(sg.Rational(el)) for el in coeffs]
#     cut_reps[cut_node].append(lg(sg.Rational([opt_cut_rhs[0]])))
    cut_rhs[cut_node] = sg.Rational([opt_cut_rhs[0]])
    hash_table[cut_check] = cut_node
    node_table[cut_node] = cut_check
    return [[full_mdl.variables.get_num() + full_mdl.linear_constraints.get_num() - 1]], 1, 1

def lift_cut(cut, group, first_group, orbits, num_tot, hash_table, node_table, full_mdl, agg_mdl):
    #### TESTING ####
#     "symm cuts"
#     new_orbits, new_cut_orbit_size = lift_cut_cy_average(cut, group, first_group, orbits, num_tot, 
#                                                  hash_table, node_table, full_mdl, agg_mdl)
    "one cut"
    new_orbits, new_cut_orbit_size = lift_cut_single_cy(cut, group, first_group, orbits, num_tot, 
                                                 hash_table, node_table, full_mdl, agg_mdl)
#     num_full_cut = copy.deepcopy(num_tot)
#     num_full_row = copy.deepcopy(num_tot)
#     num_row_orbits = 0
#     new_cut_orbit_size = {}
#     new_orbits = []
#     new_cut_orbit ={}
#     node_orbit = {}
#     agg_col_val = {}
#     agg_col_idx = {}
#     lifted_cut = [0 for col in orbits.col_orbit.keys()]
#     # TESTING TIMERS
#     gen_cut_time = 0
#     start_1 = 0
#     gen_cut_agg_orb_time = 0
#     start_2 = 0
#     add_to_full_model_time = 0
#     start_3 = 0
#     add_to_agg_model_time = 0
#     start_4 = 0
#     # GENERATE ALL CUTS IN THE ORBIT OF ORIGINAL CUT USING ORIGINAL GROUP G
#     start_1 = time.perf_counter()
#     rhs = cut[-1]
#     num_nz = 0
#     for col, coeff in enumerate(cut[:-1]):
#         for idx in range(orbits.orbit_col_start[col], orbits.orbit_col_start[col + 1]):
#             var = orbits.orbit_col[idx]
#             lifted_cut[var] = coeff
#             if abs(coeff - 0) > 1e-6:
#                 num_nz += 1
#     lift_str = ",".join(str(el) for el in lifted_cut)
#     lift_str += "," + str(rhs)
#     if hash_table.get(lift_str) != None:
#         return (None, None, new_orbits, new_cut_orbit_size)
#     cut_orbit = lg.Orbit(lg(first_group), lifted_cut, lg.Permuted)
#     gen_cut_time = time.perf_counter() - start_1
#     # LIFT THE CUTS TO THE ORIGINAL MODEL
#     cut_rows = []
#     cut_rows_sense = []
#     cut_rows_rhs = []
#     cut_rows_name = []
#     name = num_full_row
#     for cut_orb in cut_orbit:
#         start_2 = time.perf_counter()
#         ###### TESTING #######
#         cut_key = build_cut_key(cut_orb, rhs)
#         node_key = hash_table.get(cut_key)
#         full_cut_idx = full_inner_build_indices(cut_orb, rhs, num_nz)
#         full_cut_val = full_inner_build_vals(cut_orb, rhs, num_nz)
#         ##### TESTING #####
# #         cut_key = ",".join(str(el) for el in cut_orb)
# #         cut_key += "," + str(rhs)
# # #         print(cut_key)
# # #         input()
# #         node_key = hash_table.get(cut_key)
# #         full_cut_val = []
# #         full_cut_idx = []
# #         for col, coeff in enumerate(cut_orb):
# #             if (abs(coeff.sage() - 0) < 1e-6):
# #                 continue
# #             full_cut_val.append(float(coeff))
# #             full_cut_idx.append(int(col))
#         cut_rows.append(cp.SparsePair(full_cut_idx, full_cut_val))
#         cut_rows_sense.append("G")
#         cut_rows_rhs.append(float(rhs))
#         cut_rows_name.append("C%d" % name)
#         name += 1
#         add_to_full_model_time += time.perf_counter() - start_2
#         if node_orbit.get(node_key) != None:
#             continue
#         new_orbit = []
#         start_3 = time.perf_counter()
#         stab_orb = lg.Orbit(lg(group), cut_orb, lg.Permuted)
#         for orb in stab_orb:
#             orb_key = ",".join(str(el) for el in orb)
#             orb_key += "," + str(lg(rhs))
#             hash_table[orb_key] = num_full_cut
#             node_table[num_full_cut] = orb_key
#             new_orbit.append(num_full_cut)
#             node_orbit[num_full_cut] = num_row_orbits 
#             num_full_cut += 1
#         new_cut_orbit[num_row_orbits] = new_orbit[0]
#         new_cut_orbit_size[num_row_orbits] = len(new_orbit)
#         num_row_orbits += 1
#         new_orbits.append(new_orbit)
#         gen_cut_agg_orb_time += time.perf_counter() - start_3
# #     full_cut_csr = scipy.sparse.csr_matrix((full_cut_val, full_cut_idx, full_cut_start))
#     start_2 = time.perf_counter()
#     full_mdl.linear_constraints.add(lin_expr = cut_rows, senses = cut_rows_sense,
#                                        rhs = cut_rows_rhs, names = cut_rows_name)
#     add_to_full_model_time += time.perf_counter() - start_2
#     # GENERATE AGGREGATE CUTS FOR THE NEW ORBITS IN THE AGGREGATE SPACE
#     start_4 = time.perf_counter()
#     cut_rows = []
#     cut_rows_sense = []
#     cut_rows_rhs = []
#     cut_rows_name = []
#     agg_cuts = {agg_row : {} for agg_row in range(num_row_orbits)}
#     agg_cuts_rhs = {agg_row : 0 for agg_row in range(num_row_orbits)}
#     for cut_orb in cut_orbit:
#         cut_key = ",".join(str(el) for el in cut_orb)
#         cut_key += "," + str(rhs)
#         node_key = hash_table.get(cut_key)
#         node_orb = node_orbit.get(node_key)
#         for agg_col in range(orbits.num_col_orbits):
#             rep = orbits.col_orbit_rep.get(agg_col)
#             if agg_cuts.get(node_orb).get(agg_col) is None:
#                 agg_cuts[node_orb][agg_col] = 0
#             agg_cuts[node_orb][agg_col] += float(cut_orb[rep])
#         agg_cuts_rhs[node_orb] += float(rhs)
#     for agg_cut, vec in agg_cuts.items():
#         agg_cut_idx = []
#         agg_cut_val = []
#         for agg_col, val in vec.items():
#             agg_cut_idx.append(agg_col)
#             agg_cut_val.append(val)
#         cut_rows.append(cp.SparsePair(agg_cut_idx, agg_cut_val))
#         cut_rows_sense.append("G")
#         cut_rows_rhs.append(agg_cuts_rhs.get(agg_cut)) 
#         cut_rows_name.append("C%d" % node_orb)
#     agg_mdl.linear_constraints.add(lin_expr = cut_rows, senses = cut_rows_sense,
#                                 rhs = cut_rows_rhs, names = cut_rows_name)
#     add_to_agg_model_time = time.perf_counter() - start_4
#     print("\n%.2f seconds required to generate %d cuts for full model." % (gen_cut_time, len(cut_orbit)))
#     print("%.2f seconds required to generate %d cuts for aggregate model." % (gen_cut_agg_orb_time, num_row_orbits))
#     print("%.2f seconds required to add cuts to full model." % add_to_full_model_time)
#     print("%.2f seconds required to add cuts to aggregate model.\n" % add_to_agg_model_time)
# #     input()
    return (new_orbits, new_cut_orbit_size)

def numerically_stable(cut):
    if isinstance(cut, list):
        cut_len = len(cut)
        for idx in range(cut_len):
            residual = abs(cut[idx] - round(cut[idx]))
            if residual < 1e-8:
                cut[idx] = round(cut[idx])
            else:
                cut[idx] = round(cut[idx], 6)
    else:
        residual = abs(cut - round(cut))
        if residual < 1e-8:
            return round(cut)
        else:
            return round(cut, 6)
        
def write_orbits(orbits, num_col, num_row):
    for orb in orbits:
        orb.sort()
    _num_col = 0
    _num_row = 0
    for orbit in orbits:
        if orbit[0] <= num_col:
            _num_col += 1
        else:
            _num_row += 1
    with open("./sage_orbits.txt", "w") as f:
        for orbit in orbits[ : _num_col]:
            for el in orbit:
                f.write(str(el - 1) + " ")
            f.write("\n")
        for orbit in orbits[_num_col : _num_col + _num_row]:
            for el in orbit:
                f.write(str(el - 1) + " ")
            f.write("\n")
    print(orbits)
    print("Orbits Written")
    return _num_col, _num_row

def read_highs_basis(basis_file, col_basis, row_basis, 
                     agg_num_col, agg_num_row):
    col_basis.clear()
    row_basis.clear()
    with open(basis_file) as f:
        lines = f.readlines()
        for num, line in enumerate(lines):
            if num == 3:
                for col, status in enumerate(line.split(" ")[:-1]):
                    col_basis[col] = int(status)
            if num == 5:
                for row, status in enumerate(line.split(" ")[:-1]):
                    row_basis[row] = int(status)
                    
def lift_highs_basis(prev_col_basis, prev_row_basis, col_basis, row_basis, 
                     agg_col_rep, col_agg, agg_row_rep, row_agg, prev_agg_col_rep,
                     prev_col_agg, prev_agg_row_rep, prev_row_agg, agg_num_col,
                     agg_num_row, parent):
    # Set column basis
    num_linkers = len(parent)
    col_basis.clear()
    num_basic = 0
    for col, rep in agg_col_rep.items():
        old_col = prev_col_agg.get(rep)
        prev_stat = prev_col_basis.get(old_col)
        col_basis[col] = prev_stat
        if (prev_stat):
            num_basic += 1
    for link in range(num_linkers):
        col_basis[link + agg_num_col] = 0
    # Set row basis
    row_basis.clear()
    for row in agg_row_rep.keys():
        row_basis[row] = 1
#     print(prev_agg_row_rep.items())
    for old_row, rep in prev_agg_row_rep.items():
        prev_stat = prev_row_basis.get(old_row)
        row = row_agg.get(rep)
        row_basis[row] = prev_stat
#         print("old_row: %d" % old_row)
#         print("old_rep: %d" % (rep - 1))
#         print("new_row: %d" % row)
#         print("basis_stat: %d" % prev_stat)
#         input()
    for link in range(num_linkers):
        row_basis[link + agg_num_row] = 0
    for item in row_basis.values():
        if item == 1:
            num_basic += 1
    print(len(row_basis))
    print(num_basic)
    print(col_basis)
    print(row_basis)
    input()
        
def write_highs_basis(basis_file, col_basis, row_basis):
    with open(basis_file, "w") as f:
        f.write("HiGHS v1\nValid\n")
        num_col = len(col_basis)
        num_row = len(row_basis)
        f.write("# Columns %d\n" % num_col)
        for col, status in col_basis.items():
            if col == num_col - 1:
                f.write(str(status) + "\n")
                break
            f.write(str(status) + " ")
        f.write("# Rows %d\n" % num_row)
        for row, status in row_basis.items():
            if row == num_row - 1:
                f.write(str(status) + "\n")
                break
            f.write(str(status) + " ")

def write_initial_highs_basis(basis_file, agg_num_col, agg_num_row):
    with open(basis_file, "w") as f:
        f.write("HiGHS v1\nValid\n")
        f.write("# Columns %d\n" % agg_num_col)
        for col in range(agg_num_col):
            if col == agg_num_col - 1:
                f.write(str(0) + "\n")
                break
            f.write(str(0) + " ")
        f.write("# Rows %d\n" % agg_num_row)
        for row in range(agg_num_row):
            if row == agg_num_row - 1:
                f.write(str(1) + "\n")
                break
            f.write(str(1) + " ")
            
def get_old_new_orbit_linkers(agg_num_col, prev_agg_num_col, 
                              col_agg, prev_col_agg,
                              agg_col_rep, prev_agg_col_rep,
                              prev_col_basis):
    parent = []
    child = []
    if not prev_agg_num_col:
        return parent, child
    for orb, rep in agg_col_rep.items():
        prev_orb = prev_col_agg.get(rep)
        prev_orb_rep = prev_agg_col_rep.get(prev_orb)
        if orb == prev_orb:
            continue
        if rep == prev_orb_rep: 
            continue
        if prev_col_basis.get(prev_orb) != 1:
            continue
        parent_orb = col_agg.get(prev_orb_rep)
        parent.append(parent_orb)
        child.append(orb)
    return parent, child

def write_orbit_linkers(parent, child):
    with open("./orbit_linkers.txt", "w") as f:
        f.write("Parents %d\n" % len(parent))
        for el in parent:
            f.write(str(el) + " ")
        f.write("\n")
        f.write("Children %d\n" % len(child))
        for el in child:
            f.write(str(el) + " ")
        f.write("\n")
        
def pair_orbit_with_aggregate_column(orbits, num_agg_col):
    agg_col_rep = {}
    col_agg = {}
    for key, orbit in enumerate(orbits[ : num_agg_col]):
        agg_col_rep[key] = orbit[0]
        for el in orbit:
            col_agg[el] = key
    return agg_col_rep, col_agg

def pair_orbit_with_aggregate_row(orbits, num_agg_col, num_agg_row):
    agg_row_rep = {}
    row_agg = {}
    for key, orbit in enumerate(orbits[num_agg_col : num_agg_col + num_agg_row]):
        agg_row_rep[key] = orbit[0]
        for el in orbit:
            row_agg[el] = key
    return agg_row_rep, row_agg
                        
def get_stabilizer_group_orbits(group):
    group_orbits = group.orbits()
    for orb in group_orbits:
        if len(orb) > 1:
            stable_pnt = orb[0]
            stabilizer_group = group.stabilizer(stable_pnt)
#             print("New Orbits Calculated")
            return stabilizer_group, stabilizer_group.orbits()
    print("Discrete Orbits Calculated")
    return group, group.orbits()
        
def concatenate_orbits(real_orbits, cut_orbits):
    out = []
    for orb in real_orbits:
        out.append(orb)
    for orb in cut_orbits:
        out.append(orb)
    print("Orbits Concatenated")
    return out

def call_coin_cgl(lp_path):
    cgl_cut_file = "./CutGeneration/cut_txt_files/cuts.txt"
    cgl_out = sp.Popen(["./CutGeneration/generateMIPCuts",lp_path,"./sage_orbits.txt"], shell = False, 
                       stdout = sp.DEVNULL)
    cgl_return, cgl_err = cgl_out.communicate()
    print("Cgl Solved Aggregate LP")
    return cgl_return, cgl_err

def split_cut_orbits(cut_orbits, cut_vectors, cut_vectors_node, stab_group):
    t_0 = time.perf_counter()
    new_cut_orbits = []
    nodes_split = {}
#     print(cut_vectors)
#     input()
    for cut_orbit in cut_orbits:
#         print("Cut orbit:", cut_orbit)
        if len(cut_orbit) == 1: 
            new_cut_orbits.append(cut_orbit)
            continue
        for node in cut_orbit:
#             print("Focus node: %d" % node)
            if nodes_split.get(node) != None:
                continue
            cut_str = cut_vectors.get(node)
            cut_str = cut_str.split(",")
            cut_vec = [fractions.Fraction(el) for el in cut_str]
            cut_vec = [sg.Rational(el.numerator/el.denominator) for el in cut_vec]
#             print(cut_vec)
#             input()
#             print(cut_vec)
#             input()
#             cut_vec = [sg.RealNumber(el) for el in cut_vec[:-1]]
#             print(cut_vec)
#             input()
#             numerically_stable(cut_vec)
            rhs = cut_str[-1]
#             rhs = numerically_stable(rhs)
#             cut_vec = [sg.RealNumber(el) for el in cut_vec]
#             rhs = sg.RealNumber(rhs)
            orbs = lg.Orbit(lg(stab_group), cut_vec[:-1], lg.Permuted)
#             print(len(orbs))
#             print(orbs)
#             input()
            add = list()
            for orb in orbs:
                cut_key = ",".join(str(sg.Rational(el)) for el in orb)
                cut_key += "," + str(sg.Rational(rhs))
                cut_node = cut_vectors_node.get(cut_key)
                if cut_node is not None:
                    add.append(cut_node)
                    nodes_split[cut_node] = 1
                else:
                    print(cut_vectors_node)
                    print(cut_key)
                    input()
#             print(add)
#             input()
            new_cut_orbits.append(add)
#     print(new_cut_orbits)
#     input()
    t_n = time.perf_counter() - t_0
    print("Time to split cut orbits: %.2f" % t_n)
    return new_cut_orbits
            
def aggregate_cut_orbits(cut_orbits, cut_vectors, cut_vectors_node, stab_group):
    t_0 = time.perf_counter()
    new_cut_orbits = []
    nodes_aggregated = {}
#     print(cut_orbits)
#     print(cut_vectors)
#     input()
    for cut_orbit in cut_orbits:
#         print("Cut orbit:", cut_orbit)
#         if len(cut_orbit) == 1: 
#             new_cut_orbits.append(cut_orbit)
#             continue
        for node in cut_orbit:
#             print("Focus node: %d" % node)
            if nodes_aggregated.get(node) != None:
                continue
            cut_str = cut_vectors.get(node)
            cut_str = cut_str.split(",")
            cut_vec = [fractions.Fraction(el) for el in cut_str]
            cut_vec = [sg.Rational(el.numerator/el.denominator) for el in cut_vec]
#             print(cut_vec)
#             input()
#             print(cut_vec)
#             input()
#             cut_vec = [sg.RealNumber(el) for el in cut_vec[:-1]]
#             print(cut_vec)
#             input()
#             numerically_stable(cut_vec)
            rhs = cut_str[-1]
#             rhs = numerically_stable(rhs)
#             cut_vec = [sg.RealNumber(el) for el in cut_vec]
#             rhs = sg.RealNumber(rhs)
            orbs = lg.Orbit(lg(stab_group), cut_vec[:-1], lg.Permuted)
#             print(len(orbs))
#             print(orbs)
#             input()
            add = list()
            for orb in orbs:
                cut_key = ",".join(str(sg.Rational(el)) for el in orb)
                cut_key += "," + str(sg.Rational(rhs))
                cut_node = cut_vectors_node.get(cut_key)
                if cut_node is not None:
                    add.append(cut_node)
                    nodes_aggregated[cut_node] = 1
                else:
                    print(cut_vectors_node)
                    print(cut_key)
                    input()
#             print(add)
#             input()
            new_cut_orbits.append(add)
#     print(new_cut_orbits)
#     input()
    t_n = time.perf_counter() - t_0
    print("Time to aggregate cut orbits: %.2f" % t_n)
#     print(new_cut_orbits)
#     input()
    return new_cut_orbits

def generate_cut_orbits(sage_perm_group, lp_path, cut_orbits, orig_num_row,
                        prev_agg_row_rep, prev_row_agg, cut_vectors, cut_vectors_node):
    dup_cut_set = set()
    cut_rhs = []
#     cut_added_model = gp.read(lp_path)
#     orig_cols = orig_model.getVars()
    cols = cut_added_model.getVars()
    rows = cut_added_model.getConstrs()
    num_nodes = len(cols) + len(rows) + 1
    num_cut = len(prev_agg_row_rep)
    num_total_agg_row = len(prev_agg_row_rep)
    with open("./cuts.txt") as cuts_f:
        lines = cuts_f.readlines()
        for line in lines:
            new_orbits = []
#             print("num_cut %d" % num_cut)
#             print("num_node %d" % num_nodes)
            prev_agg_row_rep[num_cut] = num_nodes
            prev_row_agg[num_nodes] = num_cut
            splt = line.split(",")
            cut = []
            rhs = (sg.RealNumber(splt[-1][:-1]))
            sense = splt[-2]
            for coeff in splt[:-2]:
                cut.append(sg.RealNumber(coeff))
            if sense == "L":
                expr = sum(cut[i] * cols[i] for i in range(len(cut))) <= rhs
            elif sense == "G":
                expr = sum(cut[i] * cols[i] for i in range(len(cut))) >= rhs
            else:
                expr = sum(cut[i] * cols[i] for i in range(len(cut))) == rhs
            cut_added_model.addConstr(expr)
            new_orbits.append(num_nodes)
            cut_vectors[num_nodes] = copy.deepcopy(cut)
            cut_vectors_node[tuple(copy.deepcopy(cut))] = num_nodes
            num_nodes += 1
#             num_cut += 1
            dup_cut_set.add(tuple(cut))
#             #### Only uses partial many orbital cuts as opposed to full orbital cuts
#             cut.extend([0 for i in range(orig_num_row)])
#             for gen in sage_perm_group.gens():
#                 perm = sg.Permutation(gen.cycle_tuples(singletons=True))
#                 new_cut = perm.action(cut)
#                 new_cut = new_cut[:len(cols)]
#                 if tuple(new_cut) in dup_cut_set:
#                     continue
#                 print(new_cut)
#                 dup_cut_set.add(tuple(new_cut))
#                 if sense == "L":
#                     expr = sum(new_cut[i] * cols[i] for i in range(len(new_cut))) <= rhs
#                 elif sense == "G":
#                     expr = sum(new_cut[i] * cols[i] for i in range(len(new_cut))) >= rhs
#                 else:
#                     expr = sum(new_cut[i] * cols[i] for i in range(len(new_cut))) == rhs
#                 cut_added_model.addConstr(expr)
#                 new_orbits.append(num_nodes)
#                 prev_row_agg[num_nodes] = num_cut
#                 cut_vectors[num_nodes] = copy.deepcopy(new_cut)
#                 cut_vectors_node[tuple(copy.deepcopy(new_cut))] = num_nodes
#                 num_nodes += 1
#             num_cut += 1
              # Uses all orbital cuts that can be generated
#             new_orbits = []
#             cut.extend()
            symmetric_cuts = lg.Orbit(lg(sage_perm_group), cut, lg.Permuted)
            num_sym_cuts = len(symmetric_cuts)
#             if num_sym_cuts > 5:
#                 num_sym_cuts = 5
            for j in range(num_sym_cuts):
                sym_cut = symmetric_cuts[j]
                sym_cut = [sg.RealNumber(val) for val in sym_cut]
                if tuple(sym_cut) in dup_cut_set:
                    continue
                dup_cut_set.add(tuple(sym_cut))
                if sense == "L":
                    expr = sum(sym_cut[i] * cols[i] for i in range(len(sym_cut))) <= rhs
                elif sense == "G":
                    expr = sum(sym_cut[i] * cols[i] for i in range(len(sym_cut))) >= rhs
                else:
                    expr = sum(sym_cut[i] * cols[i] for i in range(len(sym_cut))) == rhs
                cut_added_model.addConstr(expr)
                new_orbits.append(num_nodes)
                prev_row_agg[num_nodes] = num_cut
                cut_vectors[num_nodes] = copy.deepcopy(sym_cut)
                cut_vectors_node[tuple(copy.deepcopy(sym_cut))] = num_nodes
                num_nodes += 1
            num_cut += 1
            print(new_orbits)
            input()
            cut_orbits.append(new_orbits)
    cut_added_model.write("highs_input_lp.mps")
#     orig_model.write("test.lp")
    print("Cut Orbits Generated")

def write_all_orbits(symm_lp):
    # Read initial lp in to gurobi to grab some descriptors
    col, col_name, row, row_name = read_initial_mps(symm_lp)
    # Call saucy on the lp graph
    saucy_out_file = open("./saucy_gens.txt", "w")
    saucy_out = sp.Popen(["./lp_saucy/bin/saucy2-5",symm_lp], stdout = saucy_out_file,
                                stderr = saucy_out_file, shell = False)
    saucy_return, saucy_err = saucy_out.communicate()
    sage_gens = read_saucy_gens("./saucy_gens.txt", col, row)
    # Write all the orbits of stabilizer groups to a file until the orbits are discrete
    cut_orbits = []
    sage_perm_group = PermutationGroup(sage_gens)
    real_orbits = sage_perm_group.orbits()
    orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
    write_orbits(orbits_to_write)
    stab_group, real_orbits = get_stabilizer_group_orbits(sage_perm_group)
    print(real_orbits)
    
def read_saucy_gens(saucy_file, col_idx, row_idx):
    sage_gens = []
    gen_idx = 0
    # Read in the orbit generators from saucy
    with open(saucy_file) as f:
        lines = f.readlines()
        for line in lines:
            if line[0] == "C":
                continue
            else:
                sage_gens.append([])
                orbits = re.findall(r"\((.*?)\)",line)
                for orb in orbits:
                    temp = []
                    for node in orb.split(" "):
                        if node in col_idx.keys():
                            temp.append(col_idx.get(node))
                        else:
                            temp.append(row_idx.get(node))
                    temp = tuple(temp)
                    sage_gens[gen_idx].append(temp)
                gen_idx += 1
    return sage_gens

def read_initial_mps(symm_lp):
    # Read in mps file and create dicitonary for variable names and indices 
#     orig_model = gp.read(symm_lp)
    orig_cols = orig_model.getVars()
    orig_rows = orig_model.getConstrs()
    col_idx = {col.varName : i for i, col in enumerate(orig_cols)}
    col_name = {i : col.varName for i, col in enumerate(orig_cols)}
    row_idx = {row.constrName : i + len(orig_cols) for i, row in enumerate(orig_rows)}
    row_name = {i + len(orig_cols) : row.constrName for i, row in enumerate(orig_rows)}
    return orig_model, col_idx, col_name, row_idx, row_name

def get_stabilizer_orbits(sage_orbits, sgs, act_on):
    stab_point = None
    for orbit in sage_orbits:
        if len(orbit) > 1:
            stab_point = orbit[0]
            break
    if stab_point is None:
        stab_group = PermutationGroup(sgs, domain = act_on)
        stab_orbits = stab_group.orbits()
        return stab_group, stab_orbits
    num_strong_gens = len(sgs)
    stab_gens = []
    for gen in sgs[stab_point + 1:num_strong_gens]:
        stab_gens += gen
    stab_group = PermutationGroup(stab_gens, domain = act_on)
    stab_orbits = stab_group.orbits()
    return stab_group, stab_orbits

# Get stab chain of sage group
def get_sage_stab_chain(gens, num_tot):
    group = sg.PermutationGroup(gens)
    level = 0
    stab_chain = {level : group}
    stab_group = group
    while len(stab_group.orbits()) != num_tot:
        level += 1
        stab_group, orbs = get_stabilizer_group_orbits(stab_group)
        stab_chain[level] = stab_group
    return stab_chain

# Record the orbts of each columns level by level
def record_col_orbits(orb_part, record, level):
    record[level] = orb_part.col_orbit
#     print("Col orbits by level")
#     print(record)
#     input()

# Record the column orbits by level for use in solution checking level by level
def record_col_orbit_reps(orbits, record, level):
    record[level] = {}
    num_orbs = orbits.num_col_orbits
    for i_orb in range(num_orbs):
        record[level][i_orb] = orbits.col_orbit_rep[i_orb]
#     print("Col orbit reps by level")
#     print(record)
#     input()
    
def record_previous_solution(mdl, record, level):
    record[level] = {}
    for i_col in range(mdl.variables.get_num()):
        sol_val = mdl.solution.get_values(i_col)
        record[level][i_col] = sol_val
#     print("Solution by level")
#     print(record)
#     input()

# record orb part class level by level
def record_orb_part_by_level(orb_part, record, level):
    record[level] = copy.deepcopy(orb_part)
    
# record symm group level (stabilizers as we progress)
def record_group_by_level(group, record, level):
    record[level] = group
#     print("Group by level")
#     print(record)
#     input()

def record_cut_orbit_by_level(cut_orbits, record, level):
    if record.get(level) is None:
        record[level] = []
    record[level] += cut_orbits
#     print(record)
#     input()
    
def record_cut_orbit_rep_by_level(cut_reps, record, level, start):
    if record.get(level) is None:
        record[level] = []
    for key in cut_reps.keys():
        if key <= start:
            continue
        record[level].append(key)
#     print(record)
#     input()

# Check if solution is integral
def is_solution_integral(mdl):
    for val in mdl.solution.get_values():
        if abs(round(val) - val) > 1e-9:
            return False
    return True

# Check for solutin feasibility at level k
def is_solution_level_feasible(orb_record, orb_rep_record, sol_record, level):
    print("Checking Solution")
    current_lvl = level
    current_sol = sol_record.get(current_lvl)
    for i_lvl in range(2, current_lvl):
        sol_check = {i: 0 for i in orb_record.get(i_lvl).keys()}
        for i_col, val in current_sol.items():
            i_col_rep = orb_record.get(current_lvl).get(i_col)
            old_orb = orb_rep_record.get(i_lvl).get(i_col_rep)
            sol_check[old_orb] += val
#         print(sol_check)
#         input()
        for key, val in sol_check.items():
            if abs(round(val) - val) > .01:
                return False, i_lvl
#     if level == 2:
#         return False, 1
    return True, None

# Generate the cut orbits at level k
def generate_cut_orbits_at_level(old_orbs, cut_orbs, first_group, old_group, cut_rhs, hash_table, level, single,
                                 cut_orbit_rep_by_level):
    t_0 = time.perf_counter()
    old_cut_orbs = []
    if single:
        for orb_idx, cut_orb in cut_orbs.items():
            cut = cut_orb
            old_cut_orbs.append([orb_idx])
        return old_cut_orbs
    for orb_idx, cut_orb in cut_orbs.items():
        rhs = cut_rhs[orb_idx]
        cut = cut_orb
        cut_key = ",".join(str(sg.Rational(el)) for el in cut)
        cut_key += "," + str(sg.Rational(rhs))
        hash_node = hash_table.get(cut_key)
        if hash_node in cut_orbit_rep_by_level.get(level):
#             print(hash_node)
#             input()
            continue
        unique_map = {}
        coeff_map = {}
        coeff = set()
        unique_cnt = 0
        for i_val in cut:
            if i_val not in coeff:
                coeff_map[i_val] = unique_cnt
                unique_map[unique_cnt] = i_val
                coeff.add(i_val)
                unique_cnt += 1
        for_gap = [sg.Integer(coeff_map.get(val)) for val in cut]
        first_orbit = lg.Orbit(lg(first_group), for_gap, lg.Permuted)
        old_orbit = lg.OrbitsDomain(lg(old_group), first_orbit, lg.Permuted)
        orb_lengths = lg.OrbitLengthsDomain(lg(old_group), first_orbit, lg.Permuted)
        for i_orb, old_len in enumerate(orb_lengths):
            curr_orb = []
            for i_node in range(old_len):
                this_orb = [unique_map.get(el) for el in old_orbit[i_orb][i_node].sage()]
                cut_key = ",".join(str(sg.Rational(el)) for el in this_orb)
                cut_key += "," + str(sg.Rational(rhs))
                hash_node = hash_table.get(cut_key)
                curr_orb.append(hash_node)
            old_cut_orbs.append(curr_orb)
#     print(old_cut_orbs)
#     input()
    print("Time to aggregate cut orbits: %.2f" % (time.perf_counter() - t_0))
    return old_cut_orbs

# Fucntion to call everything in a loop on LPs in a file
def orbital_cut_generation(symm_lp, best_bnd):
#     random.seed(float(0))
#     instance_name = symm_lp.split("/")[2]
#     instance_name = instance_name.split(".")[0]
#     bb = best_bnd.get(instance_name)
    bb = 3
    run_timer = [Timer()]
    run_timer[0].start()
    result_file = "output_test_symm_ocg_not_fixed_minor_iters.txt"
    cut_add_mps = "./cut_added_model_cplex.mps"
    cut_add_lp = "./cut_added_model_cplex.lp"
    cut_add_agg_lp = "./cut_added_agg_model_cplex.lp"
#     write_file = open(result_file, "a")
#     write_file.write(symm_lp + "\n")
    # write_all_orbits(symm_lp)
    # orbital_cut_generation(symm_lp)
    # READ IN THE ORIGINAL MODEL FILE WITH CPLEX
    o_mdl = cplex_read_model(symm_lp)
    o_mdl_num_col = o_mdl.variables.get_num()
    o_mdl_num_row = o_mdl.linear_constraints.get_num()
    o_mdl_num_tot = o_mdl_num_col + o_mdl_num_row
    o_mdl_col_names = o_mdl.variables.get_names()
    o_mdl_cols = {name : idx for idx, name in enumerate(o_mdl_col_names)}
    o_mdl_row_names = o_mdl.linear_constraints.get_names()
    o_mdl_rows = {name : idx + o_mdl_num_col for idx, name in enumerate(o_mdl_row_names)}
    # GET THE INITIAL ORBITS OF THE SYMM GROUP
    cut_orbits = []
    new_cut_orbits = {}
    hash_table = {}
    node_table = {}
    solution_by_level = {}
    col_orbit_by_level = {}
    col_orbit_rep_by_level = {}
    cut_orbit_reps = {}
    orb_part_by_level = {}
    group_by_level = {}
    cut_orbits_by_level = {}
    cut_orbit_reps_by_level = {}
    cut_rhs = {}
    total_full_cut = 0
    total_agg_cut = 0
    max_key = 0
    saucy_out_file = open("./saucy_gens.txt", "w")
    saucy_out = sp.Popen(["./lp_saucy/bin/saucy2-5",symm_lp], stdout = saucy_out_file,
                                stderr = saucy_out_file, shell = False)
    saucy_return, saucy_err = saucy_out.communicate()
    act_on = range(o_mdl_num_tot)
    input()
    sage_gens = read_saucy_gens("./saucy_gens.txt", o_mdl_cols, o_mdl_rows)
#     stab_chain = get_sage_stab_chain(sage_gens, o_mdl_num_tot)
#     print(sage_gens)
    sage_perm_group = PermutationGroup(sage_gens, domain = act_on)
#     return sage_perm_group
#     sage_strong_set = sage_perm_group.strong_generating_system()
    iterate = 1
    print("\n OCG Major Iteration: %d\n" % iterate)
    sage_orbits = list(list(tup) for tup in sage_perm_group.orbits())
    first_sage_orbits = copy.deepcopy(sage_orbits)
    discrete = len(sage_orbits) == o_mdl_num_tot
    orbit_part = orbital_partition(o_mdl, sage_orbits)
#     print(sage_orbits)
#     input()
    record_col_orbit_reps(orbit_part, col_orbit_rep_by_level, iterate)
    record_col_orbits(orbit_part, col_orbit_by_level, iterate)
    record_orb_part_by_level(sage_orbits, orb_part_by_level, iterate)
    record_group_by_level(sage_perm_group, group_by_level, iterate)
    first_orbit_part = orbital_partition(o_mdl, first_sage_orbits)
    agg_mdl = aggregate_A_mat(o_mdl, orbit_part)
#     first_agg_mdl = aggregate_A_mat(o_mdl, first_orbit_part)
    solve_cplex(agg_mdl)
#     agg_mdl.write("initial_agg_model.lp")
#     print("First Solution after lift")
#     agg_mdl.solution.write("first_agg_solution.txt")
#     input()
    p_obj = copy.deepcopy(agg_mdl.solution.get_objective_value())
    first_obj = copy.deepcopy(agg_mdl.solution.get_objective_value())
    write_p_obj = copy.deepcopy(p_obj)
    new_orbits, num_full_cut, num_agg_cut, feasible, level, time_out, optimal = generate_cuts_gmi(o_mdl, agg_mdl, sage_perm_group, 
                                                                             sage_perm_group, orbit_part, hash_table, 
                                                                             node_table, p_obj, iterate, total_full_cut, 
                                                                             total_agg_cut, cut_orbit_reps, cut_rhs, 
                                                                             col_orbit_rep_by_level, col_orbit_by_level, 
                                                                             solution_by_level, run_timer, bb, False)
    write_obj = agg_mdl.solution.get_objective_value()
    record_previous_solution(agg_mdl, solution_by_level, iterate)
    record_cut_orbit_by_level(new_orbits, cut_orbits_by_level, iterate)
#     temp_reps = copy.deepcopy(cut_orbit_reps)
    record_cut_orbit_rep_by_level(cut_orbit_reps, cut_orbit_reps_by_level, iterate, max_key)
    max_key = max(cut_orbit_reps.keys()) if len(cut_orbit_reps.keys()) > 0 else 0
    total_full_cut += num_full_cut
    total_agg_cut += num_agg_cut
    cut_orbits += new_orbits
    no_cut_stab_orbits = copy.deepcopy(sage_orbits)
    stab_group = copy.copy(sage_perm_group)
    if time_out or optimal:
        run_time = run_timer[0].get_run_time()
#         write_file.write("Total # cuts added to full model: %d\n" % total_full_cut)
#         write_file.write("Total # cuts added to agg model: %d\n" % total_agg_cut)
#         write_file.write("Full obj before ocg: %.4f\n" % first_obj)
#         write_file.write("Full obj at end of ocg: %.4f\n" % write_obj)
#         write_file.write("Final stabilizer level: %d\n" % iterate)
#         write_file.write("Total run time of orbital cut generation: %.2f\n\n" % run_time)
#         write_file.close()
        return
    just_swapped_levels = False
    while not discrete:
#         print("Current obj:", write_obj)
        if not just_swapped_levels:
            iterate += 1
            print("\n OCG Major Iteration: %d\n" % iterate)
            stab_group, tuple_orbits = get_stabilizer_group_orbits(stab_group)
            stab_orbits = list(list(tup) for tup in tuple_orbits)
            no_cut_stab_orbits = copy.deepcopy(stab_orbits)
            discrete = len(no_cut_stab_orbits) == o_mdl_num_tot
    #         if discrete:
    #             write_obj = agg_mdl.solution.get_objective_value()
    #             break
            cut_orbits = split_cut_orbits(cut_orbits, node_table, hash_table, stab_group)
            record_orb_part_by_level(stab_orbits, orb_part_by_level, iterate)
            stab_orbits += cut_orbits
            orbit_part = orbital_partition(o_mdl, stab_orbits)
            record_col_orbit_reps(orbit_part, col_orbit_rep_by_level, iterate)
            record_col_orbits(orbit_part, col_orbit_by_level, iterate)
            
            record_group_by_level(stab_group, group_by_level, iterate)
            agg_mdl = aggregate_A_mat(o_mdl, orbit_part)
            solve_cplex(agg_mdl)
            p_obj = copy.deepcopy(agg_mdl.solution.get_objective_value())
            write_p_obj = copy.deepcopy(p_obj)
    #         agg_mdl.solution.write("cplex_agg_solution_before.txt")
            new_orbits, num_full_cut, num_agg_cut, feasible, level, time_out, optimal = generate_cuts_gmi(o_mdl, agg_mdl, stab_group, 
                                                                                     sage_perm_group, orbit_part,
                                                                                     hash_table, node_table, p_obj, 
                                                                                     iterate, total_full_cut,
                                                                                     total_agg_cut, cut_orbit_reps,
                                                                                     cut_rhs, col_orbit_rep_by_level, 
                                                                                     col_orbit_by_level, solution_by_level,
                                                                                     run_timer, bb, True)
            write_obj = agg_mdl.solution.get_objective_value()
            record_cut_orbit_by_level(new_orbits, cut_orbits_by_level, iterate)
#             temp_reps = copy.deepcopy(cut_orbit_reps)
            record_cut_orbit_rep_by_level(cut_orbit_reps, cut_orbit_reps_by_level, iterate, max_key)
            max_key = max(cut_orbit_reps.keys())
            total_full_cut += num_full_cut
            total_agg_cut += num_agg_cut
            cut_orbits += new_orbits
            new_cut_orbits.update(cut_orbit_reps)
            if total_full_cut > 400000:
                break
            if time_out or optimal:
                break
            if not feasible:
                print("\n\n SWAPPING STABILIZER LEVELS \n \n")
                old_orbs = orb_part_by_level.get(level) + cut_orbits_by_level.get(level)
                old_group = group_by_level.get(level)
                old_cut_orbs = generate_cut_orbits_at_level(cut_orbits, new_cut_orbits, sage_perm_group, 
                                             old_group, cut_rhs, hash_table, level, 0, cut_orbit_reps_by_level)
#                 old_cut_orbs = aggregate_cut_orbits(cut_orbits, node_table, hash_table, old_group)
                combined_old_orbs = old_orbs + old_cut_orbs
#                 print(combined_old_orbs)
#                 input()
                old_orbit_part = orbital_partition(o_mdl, combined_old_orbs)
                old_agg_mdl = aggregate_A_mat(o_mdl, old_orbit_part)
    #             old_agg_mdl.write("level_swap_agg.lp")
                solve_cplex(old_agg_mdl)
    #             old_agg_mdl.solution.write("cplex_agg_solution_after_swap.txt")
                p_obj = copy.deepcopy(old_agg_mdl.solution.get_objective_value())
                temp_level = level
                new_orbits, num_full_cut, num_agg_cut, feasible, level, time_out, optimal = generate_cuts_gmi(o_mdl, old_agg_mdl, old_group, 
                                                                                     sage_perm_group, old_orbit_part,
                                                                                     hash_table, node_table, p_obj, 
                                                                                     level, total_full_cut,
                                                                                     total_agg_cut, cut_orbit_reps,
                                                                                     cut_rhs, col_orbit_rep_by_level, 
                                                                                     col_orbit_by_level, solution_by_level,
                                                                                     run_timer, bb, False)
                write_obj = old_agg_mdl.solution.get_objective_value()
                record_cut_orbit_by_level(new_orbits, cut_orbits_by_level, temp_level)
#                 temp_reps = copy.deepcopy(cut_orbit_reps)
                record_cut_orbit_rep_by_level(cut_orbit_reps, cut_orbit_reps_by_level, temp_level, max_key)
                max_key = max(cut_orbit_reps.keys())
                total_full_cut += num_full_cut
                total_agg_cut += num_agg_cut
#                 cut_orbits += new_orbits
                new_cut_orbits = {}
                if total_full_cut > 400000:
                    break
                if time_out or optimal:
                    break
                just_swapped_levels = True
                continue
            just_swapped_levels = False
            write_obj = agg_mdl.solution.get_objective_value()
        else:
            print("\n Resuming at OCG Major Iteration After Stabilizer Level Swapping: %d\n" % iterate)
#             print(no_cut_stab_orbits)
#             print(cut_orbits)
#             print(new_orbits)
#             print(stab_orbits)
#             input()
            new_cut_orbits_from_level_swapping = split_cut_orbits(new_orbits, node_table, hash_table, stab_group)
            cut_orbits += new_cut_orbits_from_level_swapping
#             record_orb_part_by_level(stab_orbits, orb_part_by_level, iterate)
            stab_orbits = no_cut_stab_orbits + cut_orbits
#             print(stab_orbits)
#             input()
            orbit_part = orbital_partition(o_mdl, stab_orbits)
#             record_col_orbit_reps(orbit_part, col_orbit_rep_by_level, iterate)
#             record_col_orbits(orbit_part, col_orbit_by_level, iterate)
            
#             record_group_by_level(stab_group, group_by_level, iterate)
            agg_mdl = aggregate_A_mat(o_mdl, orbit_part)
            solve_cplex(agg_mdl)
            p_obj = copy.deepcopy(agg_mdl.solution.get_objective_value())
            write_p_obj = copy.deepcopy(p_obj)
    #         agg_mdl.solution.write("cplex_agg_solution_before.txt")
            new_orbits, num_full_cut, num_agg_cut, feasible, level, time_out, optimal = generate_cuts_gmi(o_mdl, agg_mdl, stab_group, 
                                                                                     sage_perm_group, orbit_part,
                                                                                     hash_table, node_table, p_obj, 
                                                                                     iterate, total_full_cut,
                                                                                     total_agg_cut, cut_orbit_reps,
                                                                                     cut_rhs, col_orbit_rep_by_level, 
                                                                                     col_orbit_by_level, solution_by_level,
                                                                                     run_timer, bb, True)
            write_obj = agg_mdl.solution.get_objective_value()
            record_cut_orbit_by_level(new_orbits, cut_orbits_by_level, iterate)
#             temp_reps = copy.deepcopy(cut_orbit_reps)
            record_cut_orbit_rep_by_level(cut_orbit_reps, cut_orbit_reps_by_level, iterate, max_key)
            max_key = max(cut_orbit_reps.keys())
            total_full_cut += num_full_cut
            total_agg_cut += num_agg_cut
            cut_orbits += new_orbits
            new_cut_orbits.update(cut_orbit_reps)
            if total_full_cut > 400000:
                break
            if time_out or optimal:
                break
            if not feasible:
                print("\n\n SWAPPING STABILIZER LEVELS \n \n")
                old_orbs = orb_part_by_level.get(level) + cut_orbits_by_level.get(level)
#                 print(old_orbs)
                old_group = group_by_level.get(level)
                old_cut_orbs = generate_cut_orbits_at_level(cut_orbits, new_cut_orbits, sage_perm_group, 
                                             old_group, cut_rhs, hash_table, level, 0, cut_orbit_reps_by_level)
#                 print(old_cut_orbs)
#                 input()
#                 old_cut_orbs = aggregate_cut_orbits(cut_orbits, node_table, hash_table, old_group)
                combined_old_orbs = old_orbs + old_cut_orbs
#                 print(combined_old_orbs)
#                 input()
                old_orbit_part = orbital_partition(o_mdl, combined_old_orbs)
                old_agg_mdl = aggregate_A_mat(o_mdl, old_orbit_part)
    #             old_agg_mdl.write("level_swap_agg.lp")
                solve_cplex(old_agg_mdl)
    #             old_agg_mdl.solution.write("cplex_agg_solution_after_swap.txt")
                p_obj = copy.deepcopy(old_agg_mdl.solution.get_objective_value())
                temp_level = level
                new_orbits, num_full_cut, num_agg_cut, feasible, level, time_out, optimal = generate_cuts_gmi(o_mdl, old_agg_mdl, old_group, 
                                                                                     sage_perm_group, old_orbit_part,
                                                                                     hash_table, node_table, p_obj, 
                                                                                     level, total_full_cut,
                                                                                     total_agg_cut, cut_orbit_reps,
                                                                                     cut_rhs, col_orbit_rep_by_level, 
                                                                                     col_orbit_by_level, solution_by_level,
                                                                                     run_timer, bb, False)
                write_obj = old_agg_mdl.solution.get_objective_value()
                record_cut_orbit_by_level(new_orbits, cut_orbits_by_level, temp_level)
#                 temp_reps = copy.deepcopy(cut_orbit_reps)
                record_cut_orbit_rep_by_level(cut_orbit_reps, cut_orbit_reps_by_level, temp_level, max_key)
                max_key = max(cut_orbit_reps.keys())
                total_full_cut += num_full_cut
                total_agg_cut += num_agg_cut
#                 cut_orbits += new_orbits
                new_cut_orbits = {}
                if total_full_cut > 400000:
                    break
                if time_out or optimal:
                    break
                just_swapped_levels = True
                continue
            just_swapped_levels = False
            write_obj = agg_mdl.solution.get_objective_value()
#         write_file.write("\nMajor iteration: %d\n" % iterate)
#         write_file.write("Agg obj before cuts: %.4f\n" % write_p_obj)
#         write_file.write("Agg obj after cuts: %.4f\n" % agg_mdl.solution.get_objective_value())
#         write_file.write("# cuts added to full model: %d\n" % num_full_cut)
#         write_file.write("# cuts added to agg model: %d\n" % num_agg_cut)
#         input()
#     solve_cplex(o_mdl)
#     o_mdl.write("final_full_lp.lp")
#     write_location = "./final_full_lps/" + instance_name + ".lp"
#     o_mdl.write(write_location)
#     write_location = "./final_full_lps/" + instance_name + ".mps"
#     o_mdl.write(write_location)
#     run_time = run_timer[0].get_run_time()
#     print("Total number of cuts added to original model: %d" % total_full_cut)
#     print("Total number of cuts added to the aggregate models: %d" % total_agg_cut)
#     write_file.write("Total # cuts added to full model: %d\n" % total_full_cut)
#     write_file.write("Total # cuts added to agg model: %d\n" % total_agg_cut)
#     write_file.write("Full obj before ocg: %.4f\n" % first_obj)
#     write_file.write("Full obj at end of ocg: %.4f\n" % write_obj)
#     write_file.write("Final stabilizer level: %d\n" % iterate)
#     write_file.write("Total run time of orbital cut generation: %.2f\n\n" % run_time)
#     write_file.close()
    
        
        

In [ ]:
optimal_obj = {"cod103":-72, "cod105":-12, "cod83":-20, "cod93":-40, "codbt05":27, "codbt15":54, "codbt24":36, "codbt33":24, "codbt34":72,
               "codbt42":20, "codbt43":48, "codbt52":36, "codbt61":24, "codbt71":48, "codbt80":32, "codbt90":62, "cov1053":17, 
               "cov1054":51, "cov1075":20, "cov1076":45, "cov1174":17, "cov743":12, "cov832":11, "cov943":25, "cov954":30, "sts15":9,
               "sts27":18, "sts45":30, "sts63":45, "sts81":61, "sts243":87, "codbt02":3, "cov1052": 6, "cov632": 6, "cov945":28, "codbt03":5,
               "codbt06":75, "codbt70":16, "cod733":-98}
start = 0
f_list = []
for f in os.listdir("./margot_instances/"):
    test_type = f.split(".")[1]
    test_name = f.split(".")[0]
    if test_type == "ipynb" or test_type == "lp":
        continue
    f_list.append("./margot_instances/" + f)

f_list_sorted = sorted( f_list,
                        key =  lambda x: os.stat(x).st_size)

i = 0
for f in f_list_sorted:
    if i == 30:
        start = 1
    if start:
        print("Running file:", f)
        orbital_cut_generation(f, optimal_obj)
    i += 1
        
# orbital_cut_generation("./margot_instances/cov1174.mps", optimal_obj)

In [ ]:
# m = cplex_read_model("cut_added_agg_model_cplex_after.lp")
# m.solve()
# m.solution.write("test_sol.sol")
# print(optimal_obj.get("codbt04"))

In [ ]:
# m = cplex_read_model("./margot_instances/cov1174.mps")
# for v in m.variables.get_names():
#     m.variables.set_types(v, m.variables.type.binary)
# m.solve()

In [28]:
m = cplex_read_model("./tiny.lp")
m.write("tiny.mps")

In [29]:
orbital_cut_generation("./tiny.mps", optimal_obj)


Selected objective sense:  MINIMIZE
Selected objective  name:  obj1
Selected RHS        name:  rhs
Selected bound      name:  bnd


 OCG Major Iteration: 1

Time to create orbit part: 0.00
Time to create orbit part: 0.00
Time to add aggregate variables: 0.00
Time to add aggregate constraints: 0.00
Time to build aggregate model before cuts: 0.00
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced LP has 4 rows, 4 columns, and 12 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)
Symmetry aggregator did 4 additional substitutions.

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =            -2.666667

Dual crossover.
  Dual:  Fixed no variables.
  Primal:  Fixed no variables.
New solution before cut generation

Time to generate original gomory mixed integer cut 0.00

0.00 seconds required to generate 1 cuts for full model.
0.00 secon

KeyboardInterrupt: Interrupted by user

In [30]:
m = cplex_read_model("cut_added_agg_model_cplex_after.lp")
m.solve()
m.solution.write("cplex_agg_solution_after_cuts.txt")

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced LP has 5 rows, 4 columns, and 16 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)
Symmetry aggregator did 2 additional substitutions.

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =            -2.500000

Dual crossover.
  Dual:  Fixed no variables.
  Primal:  Fixed no variables.
